In [2]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2
import seaborn as sns
import matplotlib.pyplot as plt
import random
import zipfile
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from PIL import Image
from PIL import UnidentifiedImageError
from tqdm import tqdm 
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow_model_optimization as tfmt


ModuleNotFoundError: No module named 'tensorflow.keras'

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
print(tf.__version__)  # Debe mostrar una versión ≥ 2.0.0
print(keras.__version__)
print("OK")


2025-02-06 20:24:53.546730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738891493.567667   98868 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738891493.575426   98868 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 20:24:53.594995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: Keras cannot be imported. Check that it is installed.

In [ ]:
# Paso 1: Preprocesamiento del Dataset
# Definir el directorio donde se encuentran las imágenes del dataset
with zipfile.ZipFile('pokemon.zip', 'r') as zip_ref:
    zip_ref.extractall('pokemos')


In [9]:
dataset_dir = "pokemos/PokemonData"

In [ ]:
# Paso 1: Preprocesamiento del Dataset
# Parámetros de preprocesamiento
IMG_SIZE = (224, 224)  # Tamaño al que se redimensionarán las imágenes
BATCH_SIZE = 32  # Tamaño del lote para preprocesamiento

# Listar todas las imágenes en el directorio
image_files = []
labels = []
for root, _, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(('jpg', 'jpeg', 'png')):
            image_files.append(os.path.join(root, file))
            labels.append(os.path.basename(root))

# Convertir listas de archivos y etiquetas a tensores
image_files = tf.constant(image_files)
labels = tf.constant(labels)

# Filtrar clases con al menos 2 ejemplos
label_counts = Counter(labels.numpy())
valid_labels = [label for label, count in label_counts.items() if count >= 2]
filtered_image_files = []
filtered_labels = []
for img, label in zip(image_files.numpy(), labels.numpy()):
    if label in valid_labels:
        filtered_image_files.append(img)
        filtered_labels.append(label)

image_files = tf.constant(filtered_image_files)
labels = tf.constant(filtered_labels)

print(f"Total de imágenes después de filtrar: {len(image_files)}")


# Codificar las etiquetas para que sean valores enteros
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.numpy())

# Convertir etiquetas a tensor de TensorFlow
labels = tf.constant(labels)

# Crear un dataset de TensorFlow con las rutas de las imágenes y etiquetas
def parse_function(filename, label):
    # Leer el archivo de imagen
    img = tf.io.read_file(filename)
    # Decodificar la imagen
    img = tf.image.decode_jpeg(img, channels=3)
    # Redimensionar la imagen
    img = tf.image.resize(img, IMG_SIZE)
    # Normalizar los valores de los píxeles entre 0 y 1
    img = img / 255.0
    return img, label

dataset = tf.data.Dataset.from_tensor_slices((image_files, labels))
dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
#dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
print(f"Número de imágenes cargadas inicialmente: {len(image_files)}")
print(f"Número de etiquetas cargadas inicialmente: {len(labels)}")



In [ ]:
# Paso 2: División del Dataset
print("Dividiendo el dataset en conjuntos de entrenamiento, validación y prueba...")
dataset = dataset.shuffle(len(image_files), reshuffle_each_iteration=False)
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
print(f"Número total de muestras en el dataset: {dataset_size}")
train_size = int(0.7 * len(image_files))
val_size = int(0.2 * len(image_files))
test_size = dataset_size - train_size - val_size

print(f"Tamaño del conjunto de Entrenamiento: {train_size}")
print(f"Tamaño del conjunto de Validación: {val_size}")
print(f"Tamaño del conjunto de Prueba: {test_size}")

train_dataset = dataset.take(train_size)
val_test_dataset = dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)
"""
# Definir la función de data augmentation
def augment(image, label):
    # Aplicar transformaciones aleatorias
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.image.random_saturation(image, lower=0.9, upper=1.1)
    image = tf.image.random_hue(max_delta=0.1)
    # Asegurarse de que la imagen sigue en el rango [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# Aplicar data augmentation solo al conjunto de entrenamiento
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
"""
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"Tamaño del conjunto de Entrenamiento (número de lotes): {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Tamaño del conjunto de Validación (número de lotes): {tf.data.experimental.cardinality(val_dataset).numpy()}")
print(f"Tamaño del conjunto de Prueba (número de lotes): {tf.data.experimental.cardinality(test_dataset).numpy()}")


In [ ]:
# Paso 3: Entrenamiento del Modelo con MobileNetV2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
print("Configurando el modelo MobileNetV2 para entrenamiento...")
input_tensor = Input(shape=(224, 224, 3))  # Usando 3 canales ya que las imágenes son ahora "RGB"
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# Congelar las capas base de MobileNetV2 para transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Parámetros para pruning
prune_low_magnitude = tfmt.sparsity.keras.prune_low_magnitude
pruning_params = {
    'pruning_schedule': tfmt.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0, 
        final_sparsity=0.5, 
        begin_step=0, 
        end_step=7860
    )
}
# Añadir capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Promedio global para reducir dimensiones
x = Dense(128, activation='relu')(x)  # Capa totalmente conectada
predictions = Dense(len(valid_labels), activation='softmax')(x)  # Capa de salida

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Añadir Early Stopping para prevenir el sobreajuste
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Añadir callback para el pruning
pruning_callback = tfmt.sparsity.keras.UpdatePruningStep()

# Calcular steps_per_epoch
steps_per_epoch = np.ceil(train_size / BATCH_SIZE).astype(int)

# Entrenar el modelo
print("Entrenando el modelo...")
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping, pruning_callback]
)
